In [1]:
#Code to run the F test that compared the variances of the different treatment groups for each mitochondrial metric

In [ ]:
import pandas as pd
import scipy
from scipy import stats
import statistics
import statsmodels.stats.multitest

In [ ]:
cols = ['Treatment-naïve (mouse 260) ',
       'AC Residual (mouse 278) ', 'DTX Residual (mouse 257) ',
       'CRB Residual (mouse 268) ', 'DTX+CRB Residual (mouse 270)']

In [ ]:
variables = ["area", "length", "mbi", "mci", "perimeter", "sphericity", "volume", "width"]
df_dict = {}
for variable in variables:
    df_dict[variable] = pd.read_csv(variable + "_data.csv", header = 1)[cols]

In [ ]:
for variable in variables:
    df = df_dict[variable]
    t1s = []
    v1s = []
    t2s = []
    v2s = []
    fs = []
    cdf = []
    for i in range(len(df.columns)):
        t1 = df.columns[i]
        v1 = statistics.variance(df[t1])
        for j in range(i+1, len(df.columns)):
            t1s += [df.columns[i]]
            v1s += [v1]
            t2s += [df.columns[j]]
            t2 = df.columns[j]
            v2 = statistics.variance(df[t2])
            v2s += [v2]
        
            f = v1/v2
            fs += [f]
        
            cdf += [stats.f.cdf(f, 1, 698)]
            
            #do the opposite direction

            t1s += [t2]
            t2s += [t1]
            v1s += [v2]
            v2s += [v1]
            
            f = v2/v1
            fs += [f]
        
            cdf += [stats.f.cdf(f, 1, 698)]
            
            
            
    f_results = pd.DataFrame()
    f_results["Treatment 1"] = t1s
    f_results["Treatment 1 Variance"] = v1s
    f_results["Treatment 2"] = t2s
    f_results["Treatment 2 Variance"] = v2s
    f_results["F"] = fs
    f_results["cdf"] = cdf
    f_results["p_values (1 - cdf)"] = [1-x for x in cdf]
    f_results["p_adj"] = statsmodels.stats.multitest.multipletests(f_results["p_values (1 - cdf)"], alpha = 0.05, method = "holm")[1]
        
    f_results.to_csv(variable + "_f_results.csv")